In [2]:
from dataclasses import dataclass, field
from typing import List
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=True
from rdkit.Chem import rdmolfiles, rdmolops, AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from collections import defaultdict
import itertools
import numpy as np
import dgl.backend as F
from enum import Enum, unique
from rdkit.Chem import BondType
import dgl
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from cl_featuriser import AtomFeaturiser, BondFeaturiser

Using backend: pytorch


In [15]:
def construct_bigraph_from_mol(mol):

    g = dgl.graph(([], []), idtype=torch.int32)

    # Add nodes
    num_atoms = mol.GetNumAtoms()
    g.add_nodes(num_atoms)

    # Add edges
    src_list = []
    dst_list = []
    num_bonds = mol.GetNumBonds()
    for i in range(num_bonds):
        bond = mol.GetBondWithIdx(i)
        u = bond.GetBeginAtomIdx()
        v = bond.GetEndAtomIdx()
        src_list.extend([u, v])
        dst_list.extend([v, u])

    g.add_edges(torch.IntTensor(src_list), torch.IntTensor(dst_list))

    return g

def mol_to_graph(mol, canonical_atom_order=False):
    if mol is None:
        print('Invalid mol found')
        return None
    
    
    
    if canonical_atom_order:
        new_order = rdmolfiles.CanonicalRankAtoms(mol)
        mol = rdmolops.RenumberAtoms(mol, new_order)
    # kekulize molecule
    Chem.rdmolops.Kekulize(mol)
    # get core from whole mol
    core = MurckoScaffold.GetScaffoldForMol(mol)
    #print("core",Chem.MolToSmiles(core), mol.GetSubstructMatches(core))
    #print("mol",Chem.MolToSmiles(mol))
    sub_order = list(mol.GetSubstructMatches(core)[0])
    scaffold_list = sub_order

    for i in range(mol.GetNumAtoms()):
        if i in sub_order:
            continue
        else:
            scaffold_list.append(i)
    mol = Chem.RenumberAtoms(mol, tuple(scaffold_list))

    g = construct_bigraph_from_mol(mol)
    g_scaffold = construct_bigraph_from_mol(core)


    g.ndata.update(atom_featuriser(mol))
    g_scaffold.ndata.update(atom_featuriser(core))

    g.edata.update(bond_featuriser(mol))
    g_scaffold.edata.update(bond_featuriser(core))

    actions = []
    src, dest = g.edges()
    for i in range(core.GetNumAtoms(), mol.GetNumAtoms()):
        node_feat = g.ndata["atom_type"][i].unsqueeze(0)
        edge_dests = torch.nonzero(src==i, as_tuple=False).flatten()
        edge_srcs = dest[edge_dests]
        edge_srcs_index = torch.nonzero(edge_srcs<i, as_tuple=False).flatten()
        edge_srcs = edge_srcs[edge_srcs_index]
        edge_feat = g.edata["bond_type"][edge_dests]
        edge_feat = edge_feat[edge_srcs_index]
        #[atom type, edge type, destination]
        if len(edge_srcs) > 0:
            actions.append([
                            node_feat,
                            edge_feat,
                            edge_srcs
                            ])
        elif len(edge_srcs) == 0:
            actions.append([
                            node_feat,
                            torch.tensor([bond_featuriser.max_bond_type()]),
                            -1
                            ])
    actions.append([
                    torch.tensor([atom_featuriser.max_atom_type()]),
                    -1,
                    -1
                    ])

    return g, g_scaffold, actions

df = pd.read_csv("/BiO/pekim/GRAPHNET/data/normalised_filtered_data.csv")
gl, sl, actions = [],[], []
atom_featuriser, bond_featuriser = AtomFeaturiser(), BondFeaturiser()
for index in range(df.shape[0]):
    if index > -1:
        if index %1000 == 0:
            print(index)
        try:
            single_smi = df.loc[index].smiles
            mol = Chem.MolFromSmiles(single_smi)
            g, g_scaffold, action = mol_to_graph(mol)
            gl.append(g)
            sl.append(g_scaffold)
            actions.extend([action])
        except:
            mol = Chem.MolFromSmiles(single_smi)
            Chem.rdmolops.Kekulize(mol)
            
            core = MurckoScaffold.GetScaffoldForMol(mol)
            sub_order = list(mol.GetSubstructMatches(core))
            print(sub_order)
            break
            






0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [25]:
from dgl.data.utils import save_graphs
graph_labels = {"glabel": torch.tensor(range(len(gl)))}
save_graphs("/BiO/pekim/GRAPHNET/data/whole.bin", gl, graph_labels)
graph_labels = {"gslabel": torch.tensor(range(len(sl)))}
save_graphs("/BiO/pekim/GRAPHNET/data/scaffold.bin", sl, graph_labels)

In [27]:
from dgl.data.utils import save_graphs
graph_labels = {"glabel": torch.tensor(range(32))}
save_graphs("/BiO/pekim/GRAPHNET/data/whole_small.bin", gl[:32], graph_labels)
graph_labels = {"gslabel": torch.tensor(range(32))}
save_graphs("/BiO/pekim/GRAPHNET/data/scaffold_small.bin", sl[:32], graph_labels)

In [23]:
df_a.to_pickle("/BiO/pekim/GRAPHNET/data/normalised_filtered_data.pkl")

In [24]:
df_a[:32].to_pickle("/BiO/pekim/GRAPHNET/data/small.pkl")